In [1]:
import pandas as pd
import numpy as np 
import spacy
from tqdm import tqdm

In [2]:
fake_df = pd.read_csv("Fake.csv")
real_df = pd.read_csv("True.csv")

In [3]:
real_df.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [4]:
fake_df["label"] = "fake"
real_df["label"] = "real"

In [5]:
new_fake_df = fake_df[["title" , "label"]]
new_real_df = real_df[["title" , "label"]]

In [6]:
# cut_fake_df = new_fake_df.iloc[:5000]
# cut_real_df = new_real_df.iloc[:5000]

In [7]:
# cut_real_df.head(10)

,title,label
0,"As U.S. budget fight looms, Republicans flip t...",real
1,U.S. military to accept transgender recruits o...,real
2,Senior U.S. Republican senator: 'Let Mr. Muell...,real
3,FBI Russia probe helped by Australian diplomat...,real
4,Trump wants Postal Service to charge 'much mor...,real
5,"White House, Congress prepare for talks on spe...",real
6,"Trump says Russia probe will be fair, but time...",real
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,real
8,Trump on Twitter (Dec 28) - Global Warming,real
9,Alabama official to certify Senator-elect Jone...,real


In [37]:
news_df = pd.concat([new_real_df , new_fake_df] , axis=0 )
# Reset the index
news_df = news_df.reset_index(drop=True)

In [49]:
news_df.head(10)

,title,label,vector
0,"As U.S. budget fight looms, Republicans flip t...",0,"[-2.0529835, -0.569027, -3.1433282, 0.55970275..."
1,U.S. military to accept transgender recruits o...,0,"[-1.4419098, 2.333012, -2.0302525, 2.1737072, ..."
2,Senior U.S. Republican senator: 'Let Mr. Muell...,0,"[0.557869, 2.2574153, 0.1913623, -0.041898414,..."
3,FBI Russia probe helped by Australian diplomat...,0,"[-1.4035959, 0.07810003, -0.17810063, 2.512658..."
4,Trump wants Postal Service to charge 'much mor...,0,"[0.010546171, 1.6784946, -0.5903046, 0.4184245..."
5,"White House, Congress prepare for talks on spe...",0,"[-2.4071772, -0.9718855, -3.0410235, 1.3607991..."
6,"Trump says Russia probe will be fair, but time...",0,"[0.8482609, 0.98679125, 0.23632699, 1.0397093,..."
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,0,"[-1.6449327, -3.4080503, 1.7467283, 3.0773644,..."
8,Trump on Twitter (Dec 28) - Global Warming,0,"[-1.687013, -3.8773167, 0.4616329, 3.5332954, ..."
9,Alabama official to certify Senator-elect Jone...,0,"[-0.48848152, 1.7165813, 0.04859772, 1.860417,..."


In [39]:
news_df.label.value_counts()

fake    23481
real    21417
Name: label, dtype: int64

In [40]:
news_df["label"] = news_df["label"].replace(["fake" , "real"] , [1 , 0])

In [41]:
news_df["vector"] = 0

In [48]:
nlp = spacy.load("en_core_web_lg")

# Iterate over the DataFrame
# for index, row in tqdm(news_df.iterrows(), total=news_df.shape[0]):
#     pass

# news_df.at[index, "vector"]


# nlp = spacy.load("en_core_web_lg")

# Iterate over the DataFrame
for index, row in tqdm(news_df.iterrows(), total=news_df.shape[0]):
    news_df.at[index, "vector"] = nlp(row["title"]).vector




100%|██████████| 44898/44898 [10:26<00:00, 71.69it/s] 


In [51]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    news_df.vector.values,
    news_df.label,
    test_size=0.2,
    random_state=2022
)

In [53]:
stacked_X_train = np.stack(X_train)
stacked_X_test = np.stack(X_test)

In [55]:
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

#1. creating a KNN model object
clf = KNeighborsClassifier(n_neighbors = 2, metric = 'euclidean')

#2. fit with all_train_embeddings and y_train
clf.fit(stacked_X_train, y_train)

#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(stacked_X_test)

#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4206
           1       1.00      0.99      0.99      4774

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980

